Risk Management System - Debug & Test Notebook
Use this notebook to test each component individually

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
from datetime import date, timedelta
import warnings

warnings.filterwarnings("ignore")

# Add src to path
sys.path.append("src")
sys.path.append(".")

print("=== RISK MANAGEMENT SYSTEM DEBUG NOTEBOOK ===")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

## 1. Environment & Configuration Testing


In [ ]:
def test_environment():
    """Test environment setup and configuration loading"""
    print("\n=== TESTING ENVIRONMENT ===")

    # Test .env file loading
    from dotenv import load_dotenv

    load_dotenv()

    # Check required environment variables
    required_vars = ["EMAIL_FROM", "EMAIL_PASSWORD", "API_TOKEN"]
    env_status = {}

    for var in required_vars:
        value = os.getenv(var)
        env_status[var] = "✅ SET" if value else "❌ MISSING"
        if value:
            # Mask sensitive data for display
            masked = (
                value[:4] + "*" * (len(value) - 8) + value[-4:]
                if len(value) > 8
                else "****"
            )
            print(f"{var}: {env_status[var]} ({masked})")
        else:
            print(f"{var}: {env_status[var]}")

    # Test config file loading
    try:
        with open("config/config.yaml", "r") as f:
            config = yaml.safe_load(f)
        print("✅ Main config loaded successfully")

        with open("config/trader_accounts.yaml", "r") as f:
            traders = yaml.safe_load(f)
        print(f"✅ Trader accounts loaded: {len(traders['traders'])} traders")

        return True, config, traders
    except Exception as e:
        print(f"❌ Config loading failed: {e}")
        return False, None, None


# Run environment test
env_ok, config, traders = test_environment()

## 2. Data Loading & Validation Testing


In [ ]:
def test_data_loading():
    """Test data loading and basic validation"""
    print("\n=== TESTING DATA LOADING ===")

    try:
        from data_loader import DataLoader

        data_loader = DataLoader()

        # Test loading all trader data
        all_data = data_loader.load_all_traders_data()

        if not all_data:
            print("❌ No trader data loaded")
            return False, None, None

        print(f"✅ Loaded data for {len(all_data)} traders")

        # Validate data for each trader
        for account_id, trader_data in all_data.items():
            totals_df = trader_data["totals"]
            fills_df = trader_data["fills"]
            trader_name = trader_data["name"]

            print(f"\n📊 {trader_name} ({account_id}):")
            print(
                f"   Totals: {len(totals_df)} days, Date range: {totals_df['date'].min()} to {totals_df['date'].max()}"
            )
            print(f"   Fills: {len(fills_df)} transactions")
            print(f"   Total P&L: ${totals_df['net_pnl'].sum():,.2f}")

            # Data quality checks
            missing_dates = totals_df["date"].isnull().sum()
            missing_pnl = totals_df["net_pnl"].isnull().sum()

            if missing_dates > 0:
                print(f"   ⚠️ Missing dates: {missing_dates}")
            if missing_pnl > 0:
                print(f"   ⚠️ Missing P&L values: {missing_pnl}")

        # Create master dataset
        master_totals, master_fills = data_loader.create_master_dataset(all_data)
        print(f"\n✅ Master dataset created:")
        print(
            f"   Total records: {len(master_totals)} daily records, {len(master_fills)} fills"
        )

        return True, all_data, (master_totals, master_fills)

    except Exception as e:
        print(f"❌ Data loading failed: {e}")
        import traceback

        traceback.print_exc()
        return False, None, None


# Run data loading test
data_ok, all_data, master_data = test_data_loading()

## 3. Feature Engineering Testing


In [ ]:
def test_feature_engineering():
    """Test feature engineering pipeline"""
    print("\n=== TESTING FEATURE ENGINEERING ===")

    if not data_ok or master_data is None:
        print("❌ Cannot test features - data loading failed")
        return False, None

    try:
        from simple_feature_engineer import SimpleFeatureEngineer

        feature_engineer = SimpleFeatureEngineer()

        master_totals, master_fills = master_data

        # Engineer features
        features_df = feature_engineer.engineer_features(master_totals, master_fills)
        feature_columns = feature_engineer.get_feature_columns()

        print(f"✅ Features engineered successfully")
        print(f"   Shape: {features_df.shape}")
        print(f"   Feature count: {len(feature_columns)}")
        print(
            f"   Target distribution: {features_df['target'].value_counts().to_dict()}"
        )

        # Check for missing values
        missing_features = features_df[feature_columns].isnull().sum()
        if missing_features.sum() > 0:
            print(f"⚠️ Features with missing values:")
            for feat, count in missing_features[missing_features > 0].items():
                print(f"     {feat}: {count} missing")
        else:
            print("✅ No missing values in features")

        # Feature statistics
        print(f"\n📈 Feature Categories:")
        behavioral = [
            "trading_frequency",
            "hour_concentration",
            "symbol_diversity",
            "size_inconsistency",
            "morning_bias",
        ]
        print(
            f"   Behavioral signals: {[f for f in feature_columns if f in behavioral]}"
        )

        essential = [
            "net_pnl",
            "gross_pnl",
            "total_fees",
            "fee_ratio",
            "qty",
            "orders_count",
            "avg_fill_size",
            "net_pnl_5d_avg",
            "cum_win_rate",
            "momentum_3d",
        ]
        print(f"   Essential metrics: {[f for f in feature_columns if f in essential]}")

        temporal = ["day_of_week", "is_monday", "is_friday"]
        print(f"   Temporal features: {[f for f in feature_columns if f in temporal]}")

        return True, features_df, feature_columns

    except Exception as e:
        print(f"❌ Feature engineering failed: {e}")
        import traceback

        traceback.print_exc()
        return False, None, None


# Run feature engineering test
features_ok, features_df, feature_columns = test_feature_engineering()

## 4. Model Training & Validation Testing

In [ ]:
def test_model_training():
    """Test model training with proper validation"""
    print("\n=== TESTING MODEL TRAINING ===")

    if not features_ok or features_df is None:
        print("❌ Cannot test models - feature engineering failed")
        return False, None

    try:
        from model_trainer import ModelTrainer

        model_trainer = ModelTrainer()

        # Create time-based splits for validation
        train_df, val_df, test_df = model_trainer.create_time_splits(features_df)

        print(f"✅ Data splits created:")
        print(
            f"   Train: {len(train_df)} samples ({len(train_df['account_id'].unique())} traders)"
        )
        print(
            f"   Validation: {len(val_df)} samples ({len(val_df['account_id'].unique())} traders)"
        )
        print(
            f"   Test: {len(test_df)} samples ({len(test_df['account_id'].unique())} traders)"
        )

        # Train global model
        print("\n🔄 Training global model...")
        global_model = model_trainer.train_global_model(
            train_df, val_df, feature_columns
        )
        print("✅ Global model trained")

        # Train personal models
        print("\n🔄 Training personal models...")
        personal_models = model_trainer.train_personal_models(
            train_df, val_df, feature_columns
        )
        print(f"✅ Personal models trained: {len(personal_models)} models")

        # Train ARIMA baseline
        print("\n🔄 Training ARIMA baseline...")
        arima_models = model_trainer.train_arima_baseline(train_df, val_df)
        print(f"✅ ARIMA models trained: {len(arima_models)} models")

        # Evaluate all models
        print("\n📊 Evaluating models...")
        results = model_trainer.evaluate_models(
            test_df, global_model, personal_models, arima_models, feature_columns
        )

        print(f"✅ Model evaluation completed:")
        print(
            f"   Global Model - RMSE: {results['global']['rmse']:.4f}, MAE: {results['global']['mae']:.4f}, R2: {results['global']['r2']:.4f}"
        )
        print(
            f"   Personal Models - RMSE: {results['personal']['rmse_mean']:.4f}, MAE: {results['personal']['mae_mean']:.4f}, R2: {results['personal']['r2_mean']:.4f}"
        )
        print(
            f"   ARIMA Baseline - RMSE: {results['arima']['rmse_mean']:.4f}, MAE: {results['arima']['mae_mean']:.4f}, R2: {results['arima']['r2_mean']:.4f}"
        )

        # Save model metadata
        model_trainer.save_model_metadata(feature_columns, results)
        print("✅ Model metadata saved")

        return True, results

    except Exception as e:
        print(f"❌ Model training failed: {e}")
        import traceback

        traceback.print_exc()
        return False, None


# Run model training test
models_ok, model_results = test_model_training()

## 5. Prediction System Testing

In [ ]:
def test_prediction_system():
    """Test the prediction system"""
    print("\n=== TESTING PREDICTION SYSTEM ===")

    if not models_ok:
        print("❌ Cannot test predictions - model training failed")
        return False, None

    try:
        from predictor import RiskPredictor

        predictor = RiskPredictor()
        print("✅ Predictor initialized and models loaded")

        # Prepare data for prediction (use latest features for each trader)
        prediction_data = {}
        for account_id in all_data.keys():
            if account_id in features_df["account_id"].values:
                trader_features = features_df[features_df["account_id"] == account_id]
                prediction_data[account_id] = {
                    "features": trader_features,
                    "name": all_data[account_id]["name"],
                }

        # Generate predictions
        predictions = predictor.predict_all_traders(prediction_data)

        print(f"✅ Predictions generated for {len(predictions)} traders")

        # Display prediction summary
        risk_counts = {"High": 0, "Medium": 0, "Low": 0, "Unknown": 0}
        for pred in predictions:
            risk_level = pred.get("risk_level", "Unknown")
            risk_counts[risk_level] += 1

        print(f"📊 Risk Distribution:")
        for level, count in risk_counts.items():
            print(f"   {level}: {count} traders")

        # Show sample predictions
        print(f"\n📋 Sample Predictions:")
        for pred in predictions[:3]:
            print(
                f"   {pred['trader_name']}: {pred['risk_level']} risk, Predicted P&L: ${pred['predicted_pnl']:.2f}"
            )

        return True, predictions

    except Exception as e:
        print(f"❌ Prediction system failed: {e}")
        import traceback

        traceback.print_exc()
        return False, None


# Run prediction system test
predictions_ok, predictions = test_prediction_system()

## 6. Email Service Testing

In [ ]:
def test_email_service():
    """Test email service (without actually sending)"""
    print("\n=== TESTING EMAIL SERVICE ===")

    if not predictions_ok or not predictions:
        print("❌ Cannot test email - predictions failed")
        return False

    try:
        from email_service import EmailService

        email_service = EmailService(config)

        # Test email content creation
        html_content = email_service.create_email_content(predictions)

        print("✅ Email content generated successfully")
        print(f"   Content length: {len(html_content)} characters")

        # Save sample email for review
        with open("sample_email.html", "w") as f:
            f.write(html_content)
        print("✅ Sample email saved as 'sample_email.html'")

        # Note: Don't actually send email in testing
        print("📧 Email service ready (test send disabled)")

        return True

    except Exception as e:
        print(f"❌ Email service failed: {e}")
        import traceback

        traceback.print_exc()
        return False


# Run email service test
email_ok = test_email_service()

# %% [markdown]
# ## 7. End-to-End Integration Test


def test_integration():
    """Test the complete pipeline"""
    print("\n=== INTEGRATION TEST SUMMARY ===")

    components = {
        "Environment": env_ok,
        "Data Loading": data_ok,
        "Feature Engineering": features_ok,
        "Model Training": models_ok,
        "Prediction System": predictions_ok,
        "Email Service": email_ok,
    }

    all_ok = all(components.values())

    print(
        f"🎯 OVERALL STATUS: {'✅ READY FOR PRODUCTION' if all_ok else '❌ NEEDS FIXES'}"
    )
    print(f"\nComponent Status:")
    for component, status in components.items():
        print(f"   {component}: {'✅' if status else '❌'}")

    if all_ok:
        print(f"\n🚀 READY TO DEPLOY!")
        print(f"   - {len(all_data)} traders monitored")
        print(f"   - {len(feature_columns)} features engineered")
        print(
            f"   - Global + {len(personal_models) if models_ok else 0} personal models trained"
        )
        print(f"   - Email reporting configured")

        # Save integration test results
        test_results = {
            "timestamp": pd.Timestamp.now().isoformat(),
            "component_status": components,
            "trader_count": len(all_data) if all_data else 0,
            "feature_count": len(feature_columns) if feature_columns else 0,
            "model_performance": model_results if models_ok else None,
        }

        import json

        with open("integration_test_results.json", "w") as f:
            json.dump(test_results, f, indent=2, default=str)
        print("✅ Test results saved to 'integration_test_results.json'")

    else:
        print(f"\n🔧 FIXES NEEDED:")
        for component, status in components.items():
            if not status:
                print(f"   - Fix {component}")

    return all_ok


# Run integration test
integration_ok = test_integration()

# %% [markdown]
# ## 8. Manual Testing & Debugging Functions


def debug_trader_data(account_id):
    """Debug specific trader data"""
    if not all_data or account_id not in all_data:
        print(f"Trader {account_id} not found")
        return

    trader = all_data[account_id]
    print(f"\n=== DEBUG: {trader['name']} ({account_id}) ===")

    totals = trader["totals"]
    fills = trader["fills"]

    print(f"Totals Data:")
    print(f"  Shape: {totals.shape}")
    print(f"  Date range: {totals['date'].min()} to {totals['date'].max()}")
    print(f"  Total P&L: ${totals['net_pnl'].sum():,.2f}")
    print(f"  Average daily P&L: ${totals['net_pnl'].mean():.2f}")
    print(f"  Win rate: {(totals['net_pnl'] > 0).mean():.1%}")

    print(f"\nFills Data:")
    print(f"  Shape: {fills.shape}")
    print(f"  Unique symbols: {fills['symbol'].nunique()}")
    print(f"  Average trade size: ${(fills['qty'] * fills['price']).mean():,.2f}")

    return totals, fills


def debug_features(account_id):
    """Debug features for specific trader"""
    if not features_ok:
        print("Features not available")
        return

    trader_features = features_df[features_df["account_id"] == account_id]
    if trader_features.empty:
        print(f"No features found for trader {account_id}")
        return

    print(f"\n=== FEATURES DEBUG: {account_id} ===")
    print(f"Feature samples: {len(trader_features)}")
    print(f"Target distribution: {trader_features['target'].value_counts().to_dict()}")

    # Show latest features
    latest = trader_features.iloc[-1]
    print(f"\nLatest Features:")
    for col in feature_columns:
        print(f"  {col}: {latest[col]:.4f}")

    return trader_features


def debug_prediction(account_id):
    """Debug prediction for specific trader"""
    if not predictions_ok:
        print("Predictions not available")
        return

    pred = next((p for p in predictions if p["account_id"] == account_id), None)
    if not pred:
        print(f"No prediction found for trader {account_id}")
        return

    print(f"\n=== PREDICTION DEBUG: {account_id} ===")
    for key, value in pred.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")

    return pred


# Example usage functions
print(f"\n=== MANUAL DEBUGGING FUNCTIONS AVAILABLE ===")
print(f"- debug_trader_data('account_id')")
print(f"- debug_features('account_id')")
print(f"- debug_prediction('account_id')")

if all_data:
    first_trader = list(all_data.keys())[0]
    print(f"\nExample: debug_trader_data('{first_trader}')")

print(f"\n=== NOTEBOOK READY FOR DEBUGGING ===")